# 5. Regresja

Regrasja polega na stworzeniu modelu opisującego zależność między zmiennymi. Posiadając taki model możemy przewidywać wartość  jednej zmiennej ciągłej na podstawie innych zmienneych. 

Poniższy notatnik demonstruje:
1. Metody regresji liniowej i wielomianowej z użyciem pakietu scikit-learn
2. Tworzenie modelu (fit), predykcję wartości (predict) oraz ocenę jakości predykcji (MSE. $R^2$-score)
3. Problem przeuczenia (overfitting) oraz sposoby redukcji tego problemu za pomocą regularyzacji


## Generowanie danych treningowych

Stworzymy najpierw sztuczne dane, na których zbudujemy model regresyjny


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Zdefinujmy funkcję `true_function()`, która posłuży do wygenerowania danych do których będziemy dopasowywali nasz model. 

$$ f(x) = \cos \left(1.5 \cdot \pi \cdot  x\right) $$

In [ ]:
def true_function(x):
    return np.cos(1.5 * np.pi * x)

Wygenerujmy próbkę danych `x` oraz wyznaczmy wartości funkcji `y` dla tych danych.   
Będziemy szukać zależności `x ~ y`


Zainicjalizujmy najpierw ziarno dla funkcji losującej, aby za każdym razem otrzymać *te same* ale losowe dane. Często się to przydaje, gdy operujemy na losowych danych, ale chcemy zagwarantować powtarzalność wyników. 

In [ ]:
np.random.seed(0)

In [ ]:
n_samples = 30
x = np.sort(np.random.rand(n_samples, 1))
y = true_function(x) 

Narysujmy tą funkcję oraz wygenerowane punkty.

In [ ]:
#przedział do 'gęstego' narysowania funkcji,
x_range = np.linspace(0, 1, 100)

# operacja reshape tworzy transpozycje wektora, otrzymujemy macierzy o wymiarach (100, 1), 
# taka postać jest wymagana później przez model regresyjny LinearRegression()
x_range = np.reshape(x_range, (-1, 1))

plt.plot(x_range, true_function(x_range), label='funkcja prawdziwa')
plt.plot(x, y, 'ro', label='Punkty treningowe')
plt.legend()
plt.show()

Dodajmy trochę szumu do danych aby utrudnić problem dopasowania modelu w naszym przykładzie. 

In [ ]:
y = true_function(x) + np.random.randn(n_samples, 1) * 0.1

plt.plot(x_range, true_function(x_range))

plt.plot(x, y, 'ro',)
plt.legend(['funkcja prawdziwa','punkty uczące'])
plt.show()

## Model liniowy

Dopasujmy do naszych danych model liniowy dla jednej zmiennej 

$$ f(x) = x w_1 + w_0 $$

Zadanie polega na znalezieniu takich parametrów $w_1$ i $w_0$ definiujących linię aby błąd popełniany przez tą funkcję był jak najmniejszy, tzn. chcemy aby punkty danych leżały w jak najmniejszej odległości od dopasowanej linii.  Funkcja kosztu (błędu) to suma kwadratów różnic między punktem prawdziwym $y_i$ a tym uzyskanym z modelu $f(x_i)$:

$$L(y, f(x,w)) = \sum_{i=1}^N \|y_i - f(x_i)\|^2,$$

Zadanie polega więc na znalezieniu takich $w_1$ i $w_0$, które minimalizują wartość błędu $L$. W celu wyznaczenia tego minimum wykorzystamy algorytm regresji liniowej dostępny w pakiecie scikit-learn [LinearRegression()](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html).

## Regresja wielu zmiennych 

Ogólna postać modelu regrasji liniowej wielowymiarowej  

$$f(\mathbf{x}) = x_1 w_1 + x_2 w_2 + \ldots + x_k w_k + w_0= \mathbf{x}^T \mathbf{w} + w_0$$

gdzie dla $k$ zmiennych $\mathbf{w}$ jest wektorem $[w_1, w_2, \ldots, w_k]$ współczynników i wraz z wyrazem wolnym $w_0$ są szukanymi wartościami określającymi liniową zależność w przestrzeni $\mathbb{R}^k$ 

## Liniowa regresja w scikit-learn

In [ ]:
from sklearn.linear_model import LinearRegression

# Tworzymy obiekt realizujący liniową regresję
regr = LinearRegression()

Metoda ``fit(x, y)`` modelu predykcyjnego uruchamia trening modelu

In [ ]:
# Dopasowanie modelu do danych (poszukiwanie minimum funkcji błedu)
regr.fit(x, y)

# Wyznaczone wspólczynniki regresji
a = regr.coef_[0]           # wspólczynnik kierunkowy  
b = regr.intercept_         # wspólczynnik przecięcia

print("f(x) = %f x + %f" % (a, b))

## Predykcja

Metoda [predict()](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression.predict) pozwala wyznaczyć przewidywaną wartość zmiennej wyjściowej dla dowonych wartości wejściowych `x` 

In [ ]:
regr.predict([[0.5]])

In [ ]:
# to samo
print(a * 0.5 + b)  

In [ ]:
y_pred = regr.predict(x_range)

plt.plot(x_range, true_function(x_range), label='funkcja prawdziwa')
plt.plot(x_range, y_pred, 'g--', label='regresja liniowa')
plt.plot(x, y, 'ro', label='punkty uczące')
plt.legend()
plt.show()

In [ ]:
# co sprowadza się w tym przypadku do operacji
y_lin = a * x_range + b      # model liniowy zalezosci X ~ Y

np.alltrue(y_pred == y_lin)

## Regresja wielomianowa

Model liniowy w tym przypadku jest daleki od ideału. Spróbujmy dopasować do danych funkcję wielomianową.

$$ f(x) = x^k w_k + x^{k-1} w_{k-1} + \ldots + x w_1 + w_0 $$

gdzie $k$ określa stopień wielomianu.  
Dla $k=1$ otrzymamy znowu model liniowy, dla $k=2$ parabolę, itd.

Jak widać sprowadza się to do znalezienia wspólczynników $w_i$ liniowego modelu dla $k$ zmiennych, gdzie zmienne są postaci $x, x^2, \ldots, x^k$.

Wykorzystajmy funkcję [PolynomialFeatures()](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) do przekształcenia danych $x$ do takiej postaci.

### Transformacja wielomianowa w scikit-learn

Stwórzmy model liniowy dla zmiennych $x^2$ i $x$, co odpowiada dopasowaniu wielomianu 2 stopnia $$f(x) = w_2 x^2 + w_1 x + w_0$$

[PolynomialFeatures()](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) jest transformatorem danych, API takich modeli w scikit-learn wygląda tak: 
* ``fit(x)`` - trening (dopasowanie) modelu transformacji na danych treningowych
* ``transform(x)`` - transformacja (nowych) danych za pomocą modelu uzyskanego w treningu
* ``fit_transform(x)`` - trening i transformacja danych treningowych 

In [ ]:
from sklearn.preprocessing import PolynomialFeatures 

# regresja wielomianem stopnia 2 
poly2 = PolynomialFeatures(degree = 2) 
x_poly2 = poly2.fit_transform(x)

In [ ]:
# sprawdźmy 
x_tmp =  np.hstack( [ np.ones(x.shape), x, x*x] )   #  x zamienione na 1, x, x^2
np.alltrue(x_tmp == x_poly2)

In [ ]:
regr2 = LinearRegression()
regr2.fit(x_poly2, y)

plt.plot(x_range, true_function(x_range), label='prawdziwa funkcja')
plt.plot(x_range, regr2.predict(poly2.transform(x_range)), 'g--', label='regresja wielomianem stopnia 2')
plt.plot(x, y, 'ro', label='punkty treningowe')
plt.legend()
plt.show()

print('%d współczynniki regresji: %s'  % ( len(regr2.coef_[0]), str(regr2.coef_[0])))

Wynik wygląda lepiej. Spróbujmy dopasować wielomian wyższego rzędu.

### Dopasowanie wielomianu stopnia 3

In [ ]:
poly3 = PolynomialFeatures(degree = 3) 
x_poly3 = poly3.fit_transform(x) 

regr3 = LinearRegression()
regr3.fit(x_poly3, y)

plt.figure(figsize=(10,8))

plt.plot(x_range, true_function(x_range), label='prawdziwa funkcja')
plt.plot(x_range, regr2.predict(poly2.transform(x_range)), 'g--', label='regresja wielomianowa d=2')
plt.plot(x_range, regr3.predict(poly3.transform(x_range)), 'm--', label='regresja wielomianowa d=3')
plt.plot(x, y, 'ro', label='punkty uczące')
plt.legend()
plt.show()

print('%d współczynniki regresji: %s'  % ( len(regr3.coef_[0]), str(regr3.coef_[0])))

## Przeuczenie

Wygląda na to, że jeśli będziemy dopasowywać wielomian coraz wyższego stopnia, otrzymamy coraz lepszą aproksymację danych. Niemniej jednak zwiększanie stopnia wielomianu, a co za tym idzie 'poziomu skomplikowania' naszego modelu, będzie powodować zbyt duże dopasowanie się do danych i zbyt małe możliwości generalizacji modelu. W szczególności możemy mówić o dwóch fenomenach jeżeli uczymy się parametrów modelu z danych:

* **underfit** - model jest zbyt 'prosty' aby uchwycić zależności między danymi
* **overfit** (przeczenie)  - parametry naszego modelu 'nauczyły' się szumu, model jest zbyt dopasowany do danych treningowych

![image.png](attachment:image.png)


Istnieje zatem całkiem duże prawdopodobieństwo, że zwiększając stopień wielomianu dopasowaliśmy wielomian do szumu. Aby to zilustrować weźmy wielomian 16-go stopnia:

In [ ]:
poly16 = PolynomialFeatures(degree = 16)
x_poly16 = poly16.fit_transform(x)

regr16 = LinearRegression() 
regr16.fit(x_poly16, y)

plt.figure(figsize=(10,8))
plt.plot(x_range, true_function(x_range), label='prawdziwa funkcja')
plt.plot(x_range, regr16.predict(poly16.fit_transform(x_range)), 'b--', label='regresja wielomianowa d=16 (przeuczenie)')
plt.plot(x, y, 'ro', label='punkty uczące')
plt.ylim((-1.2, 1.7))
plt.legend()
plt.show()

print('%d współczynników regresji: %s'  % ( len(regr16.coef_[0]), str(regr16.coef_[0])))

## Ocena modelu
### Błąd MSE

Podstawową miarą służącą do oceny modelu regresyjnego jest średnia wartość funkcji kosztu, czyli tzw. **błąd średniokwadratowy MSE** (*mean squared error*):

$$ MSE = \frac{1}{n} \sum_{i=1}^n \left( y_i - f(x_i) \right)^2 $$

W przypadku idealnego dopasowania, gdy wszystkie $y_i$ są bezbłędnie dopasowane przez funkcję $f(x_i)$, bład MSE ma wartość 0. 

### Miara $R^2$

Drugą powszechnie używaną miarą oceny jest **współczynnik determinacji $R^2$**:

$$ R^2 = \frac{\sum_{i=1}^n \left( f(x_i) - \bar{y}_i\right)^2}{\sum_{i=1}^n \left( y_i - \bar{y}_i\right)^2} > 0$$

gdzie $\bar{y}$ to średnia wartość odpowiedzi $y_i$ dla wszystkich punktów w danych $i=1,\ldots,n$.  

Im większa wartość współczynnika $R^2$ tym lepsze dopasowanie $f(x)$ do danych.

W przypadku bezbłędnego dopasowania $R^2=1$.

### Meryki oceny w scikit-learn

Pakiet ``sklearn.metrics`` zawiera szereg metryk to oceny modeli, m. in. metryki do oceny modeli regresyjnych:
* funkcja ``mean_squared_error(y_true, y_pred)``
* funkcja ``r2_score(y_true, y_pred)``

Modele predykcyjne posiadają też metodę ``score(x, y)`` do wyznaczania oceny (dla ``LinearRegression()`` domyślnie $R^2$) 

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error

y_pred = regr.predict(x)
y_regr2_pred = regr2.predict(poly2.transform(x))
y_regr3_pred = regr3.predict(poly3.transform(x))
y_regr16_pred = regr16.predict(poly16.transform(x))

import pandas as pd

columns = ['Model', 'MSE', 'R2']
data = pd.DataFrame(columns=columns)
data.loc[0] = ['linear',   mean_squared_error(y, y_pred),        r2_score(y, y_pred)        ] 
data.loc[1] = ['poly 2',   mean_squared_error(y, y_regr2_pred),  r2_score(y, y_regr2_pred)  ] 
data.loc[2] = ['poly 3',   mean_squared_error(y, y_regr3_pred),  r2_score(y, y_regr3_pred)  ] 
data.loc[3] = ['poly 16',  mean_squared_error(y, y_regr16_pred), r2_score(y, y_regr16_pred) ] 

data

In [ ]:
# R2-score za pomoca metody score() modelu
regr.score(x, y)

In [ ]:
import seaborn as sb
sb.barplot(data=data, x='Model', y='MSE')

In [ ]:
sb.barplot(data=data, x='Model', y='R2')

Obliczanie błędu MSE na danych `(X, Y)`, na których dokonano dopasowania modelu nie jest wiarygodną oceną. Widać, że model najbardziej złożony i przeuczony ma najmniejszy błąd MSE. Znając prawdziwą funkcję, możemy obliczyć błąd popełniany przez model, jednak w rzeczywistych zastosowaniach nie mamy dostępu do prawdziwej funkcji, gdyż to właśnie tą funkcję próbujemy odkryć. Potrzebujemy danych do testowania. W przypadku, gdy mamy do dyspozycji dużo danych wejściowych możemy je podzielić na część **treningową** oraz część **testową**. Wówczas model dopasowany na zbiorze treningowym może zostać oceniony na części testowej, która nie została użyta do stworzenia modelu. Jeżeli model regresyjny będzie posiadał mały błąd również na danych testowych to znaczy, że znaleźliśmy ogólną regułę, prawdziwą dla tego typu danych (tzn.model generalizuje).

### Ocena na danych testowych

Załóżmy, że posiadamy osobny fragment naszych wygenerowanych danych, który nie brał udziału w tworzeniu modelu. W naszym przypadku wygenerujemy nowe obserwacje, które posłużą w roli zbioru testowego.

In [ ]:
x_test = np.sort(np.random.rand(100, 1))
y_test = true_function(x_test) + np.random.randn(100, 1) * 0.1

Dokonajmy oceny na zbiorze tesowym

In [ ]:
y_test_pred = regr.predict(x_test)
y_regr2_test_pred = regr2.predict(poly2.fit_transform(x_test))
y_regr3_test_pred = regr3.predict(poly3.fit_transform(x_test))
y_regr16_test_pred = regr16.predict(poly16.fit_transform(x_test))

mse_test = [ 
     mean_squared_error(y_test, y_test_pred),
     mean_squared_error(y_test, y_regr2_test_pred),
     mean_squared_error(y_test, y_regr3_test_pred),
     mean_squared_error(y_test, y_regr16_test_pred)
]

data['MSE test'] = mse_test

sb.barplot(data=data, x='Model', y='MSE test')

data

Jak można było się spodziewać model o największej złożoności (wielomian stopnia 16) ma największy błąd na zbiorze testowym. Na zbiorze treningowym jego błąd był bardzo mały, to znaczy, że model jest przeuczony i zbytnio dopasował się do danych w zbiorze uczącym. Najlepszą generalizację (zdolność do popranego przewidywania `y` na nowych danych) w tym wypadku miała regresja wielomianowa stopnia 3.


## Regularyzacja

Metody regularyzacji mają za zadanie zminimalizować ryzyko przeuczenia modelu. W przypadku regresji powszechnie stosowanym podejściem jest dodanie do funkcji kosztu $L$ dodatkowego członu, który będzie wymuszał zmniejszanie wartości wag $w_i$. W szczególnym wypadku, gdy waga zmaleje do zera $w_i=0$ można uznać, że zmienna $i$-ta jest nieistotna dla modelowanej zależności (czynnik $w_i x_{ij}$ zanika). W ten sposób wymuszając zmniejszanie wartości wag w trakcie optymalizacji preferowane są modele o mniejszej złożoności (część wag $w_i$ zaniknie lub będzie miała małe wartości).

### Regresja grzbietowa

Regresja grzbietowa (*Ridge regression*) polega poszukiwaniu parametrów $w_i$ poprzez minimalizację poniższej funkcji kosztu
$$L_{\textrm{ridge}} = \sum_{i=1}^N \|y_i - f(x_i)\|^2 + \alpha \|w\|^2 $$

gdzie $\alpha > 0$ jest parametrem określającym siłę regularyzacji.  
Dla $\alpha=0$ człon regularyzacyjny zanika i regresja grzbietowa zamienia się w zwykła regresję liniową. Zazwyczaj $\alpha$ jest małą wartością dodatnią np. 0.01.
Minimalizacja powyższej funkcji kosztu dąży do minimalizacji błędu MSE (pierwszy człon) oraz do zanikania wartości wag (drugi człon).

Spróbujmy zobaczyć jaki wynik uzyskamy dla regresji wielomianowej stopnia 16 w naszym przypadku, gdy zastosujemy regularyzację.

In [ ]:
from sklearn import linear_model

ridge = linear_model.Ridge(alpha=0.02)
ridge.fit(x_poly16, y)

plt.figure(figsize=(10, 8))

plt.plot(x_range, true_function(x_range), label='prawdziwa funkcja')
plt.plot(x_range, ridge.predict(poly16.fit_transform(x_range)), 'b--', label='ridge (alpha=0.02)')
# plt.plot(x_range, regr16.predict(poly16.fit_transform(x_range)), 'g--', label='poly 16')
plt.plot(x, y, 'ro', label='punkty treningowe')
plt.ylim((-1.2, 1.7))
plt.legend()
plt.show()


In [ ]:
y_ridge_pred_test = ridge.predict(poly16.fit_transform(x_test))
y_ridge_pred_train = ridge.predict(poly16.fit_transform(x))
                             
data.loc[4] = ['ridge',  
               mean_squared_error(y, y_ridge_pred_train), 
               r2_score(y, y_ridge_pred_train),
               mean_squared_error(y_test, y_ridge_pred_test)] 
data

In [ ]:
sb.barplot(data=data, x='Model', y='MSE test')

Błąd na zbiorze testowym wskazuje, że dopasowanie wielomianu stopnia 16 do naszego przykładu z zastosowaniem regularyzacji grzbietowej osiągnął zbliżony wynik do wielomianu stopnia 3. 

Zobaczmy jak wyglądają wartości bezwzględne uzyskanych parametrów $w_i$ dla modelu z regularyzacją i bez regularyzacji.

In [ ]:
print('Suma wartości wag modelu z regularyzacją grzbietową:', np.absolute(ridge.coef_[0]).sum())
print('Suma wartości wag modelu bez regularyzacji:', np.absolute(regr16.coef_[0]).sum())

plt.bar(range(len(regr16.coef_[0])), np.absolute(regr16.coef_[0]))
plt.title('Współczynniki regresji bez regularyzacji')
plt.show()
plt.bar(range(len(ridge.coef_[0])), np.absolute(ridge.coef_[0]))
plt.title('Współczynniki regresji grzbietowej (z regularyzacją)')
plt.show()

### Regresja Lasso

Regresja metodą Lasso jest bardzo podobna do regresji grzbietowej, jednak zamiast normy $L_2$ w czynniku regularyzacyjnym występuje norma $L_1$:

$$L_{\textrm{lasso}} = \sum_{i=1}^N \|y_i - f(x_i)\|^2 + \alpha \|w\|_1 $$

Tego typu regularyzacja również preferuje rozwiązania z małymi wartościami wag, jednak w odróżnieniu od regresji grzbietowej otrzymana reprezentacja jest "rzadsza", tzn. większa liczba  spółczynników $w_i$ jest redukowanych do 0.

Wykonajmy ponownie dopasowanie wielomianem stopnia 16 tym razem z regularyzacją Lasso

In [ ]:
from sklearn import linear_model

lasso = linear_model.Lasso(alpha=0.01)
lasso.fit(x_poly16, y)

plt.figure(figsize=(10, 8))

plt.plot(x_range, true_function(x_range), label='funkcja prawdziwa')
plt.plot(x_range, lasso.predict(poly16.fit_transform(x_range)), 'b--', label='regresja lasso (alpha=0.01)')
# plt.plot(x_range, regr16.predict(poly16.fit_transform(x_range)), 'g--', label='poly 16')
plt.plot(x, y, 'ro', label='punkty treningowe')
plt.ylim((-1.2, 1.7))
plt.legend()
plt.show()                        



In [ ]:
y_pred_train_lasso = lasso.predict(poly16.fit_transform(x))
y_pred_test_lasso = lasso.predict(poly16.fit_transform(x_test))

data.loc[5] = ['lasso',  
               mean_squared_error(y, y_pred_train_lasso), 
               r2_score(y, y_pred_train_lasso),
               mean_squared_error(y_test, y_pred_test_lasso)] 

sb.barplot(data=data, x='Model', y='MSE test')
data

Uzyskany wynik jest gorszy od modelu regresji grzbietowej ale jest też o wiele lepszy od modelu, który nie był poddany regularyzacji. Odpowiedni dobór siły regularyzacji $\alpha$ powinien pozwolić uzyskać wynik zbliżony a może i lepszy od regresji grzbietowej. 

Porównanie wartości współczynników uzyskanych obiema metodami regularyzacji.

In [ ]:
print('Współczynniki modelu regresji grzbietowej')
print(ridge.coef_[0])

plt.bar(range(len(ridge.coef_[0])), np.absolute(ridge.coef_[0]))
plt.title('Regresja grzebietowa')
plt.show()

print('Współczynniki regresji modelu lasso')
print(lasso.coef_)

plt.bar(range(len(lasso.coef_)), np.absolute(lasso.coef_))
plt.title('Regresja lasso')
plt.show()

W przypadku regularyzacji Lasso tylko 3 współczynniki są niezerowe, tzn. tylko te 3 zmienne wystarczą do uzyskania predykcji wartości `y` przy zachowaniu niewielkiego błędu.

# Zestaw danych Housing

Zbiór danych cen mieszkań w Bostonie (506 wierszy i 14 kolumn). Każdy wiersz reprezentuje dom znajdujący się w Bostonie w stanie Massachusetts w 1978 r.  
Celem jest oszacowania średniej wartości domu (MEDV). 

Źródło: [https://archive.ics.uci.edu/ml/datasets/Housing](https://archive.ics.uci.edu/ml/datasets/Housing)

Atrybuty:
    
<pre>
1. CRIM      współczynnik przestępczości per capita na każde miasteczko
2. ZN        odsetek działek przekraczających 25 000 stóp kwadratowych (≈ 2533 metrów kwadratowych).
3. INDUS     odsetek terenów przeznaczonych na przemysł niedetaliczny na każde miasteczko
4. CHAS      zmienna zerojedynkowa określająca rzekę Charles (przyjmuje wartość 1, gdy na danym terenie znajduje się koryto      
             rzeki)
5. NOX       stężenie tlenków azotu (w częściach na 10 milionów)
6. RM        średnia liczba pomieszczeń na dom
7. AGE       odsetek zamieszkałych budynków wybudowanych przed 1940 rokiem
8. DIS       ważona odległość do pięciu bostońskich urzędów pracy
9. RAD       wskaźnik dostępności do głównych arterii komunikacyjnych
10. TAX      pełna wartość podatku od nieruchomości na każde 10 000 dolarów
11. PTRATIO  stosunek liczby uczniów do nauczycieli na każde miasteczko
12. B        parametr wyliczany ze wzoru 1000(Bk - 0.63)^2, gdzie Bk oznacza odsetek osób pochodzenia afroamerykańskiego 
             zamieszkujących dane miasteczko 
13. LSTAT    odsetek ubogiej części społeczeństwa
14. MEDV     mediana wartości zamieszkanych domów wyrażona w tysiącach dolarów
</pre>

In [ ]:
import pandas as pd

df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data', 
                 header=None, sep='\s+')

df.columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 
              'NOX', 'RM', 'AGE', 'DIS', 'RAD', 
              'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
df.head()

## Podstawowe informacje o danych

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AGE     442 non-null    int64  
 1   SEX     442 non-null    int64  
 2   BMI     442 non-null    float64
 3   BP      442 non-null    float64
 4   S1      442 non-null    int64  
 5   S2      442 non-null    float64
 6   S3      442 non-null    float64
 7   S4      442 non-null    float64
 8   S5      442 non-null    float64
 9   S6      442 non-null    int64  
 10  Y       442 non-null    int64  
dtypes: float64(6), int64(5)
memory usage: 38.1 KB


In [ ]:
plt.figure(figsize=(15,5))
df.boxplot()
display(df.describe())

NameError: ignored

In [ ]:
df.isna().any()

## Wsółczynnik korelacji liniowej

Korelacja liniowa [Pearsona](https://pl.wikipedia.org/wiki/Wsp%C3%B3%C5%82czynnik_korelacji_Pearsona) 
$$
r_{x y}=\frac{\sum_{i=1}^{n}\left(x_{i}-\bar{x}\right)\left(y_{i}-\bar{y}\right)}{\sqrt{\sum_{i=1}^{n}\left(x_{i}-\bar{x}\right)^{2}} \sqrt{\sum_{i=1}^{n}\left(y_{i}-\bar{y}\right)^{2}}}
$$

Wartość bliska 1 lub -1 wskazuje o silnej korelacji linowej


In [ ]:
#Sort correlations
correlations = df.corr()
correlations

In [ ]:
import seaborn as sns

plt.figure(figsize=(8, 8))
sns.heatmap(correlations, cmap='coolwarm', vmin=-1, vmax=1, square=True)

In [ ]:
# Wspólczynniki korelacji dla zmiennej MEDV

corr_mdev = correlations['MEDV'].sort_values()
corr_mdev

## Wizualizowanie ważnych elementów zestawu danych

Wybieżmy zmienne o wysokim wspólczynniku korelacji (>0.5)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

cols = ['LSTAT', 'PTRATIO', 'RM', 'MEDV']

sns.pairplot(df[cols], height=2.5)
plt.tight_layout()
plt.show()

In [ ]:
cm = df[cols].corr()
# sns.set(font_scale=1.0)
plt.figure(figsize=(10,10))
hm = sns.heatmap(cm, 
            cbar=True,
            annot=True, 
            square=True,
            fmt='.2f',
            annot_kws={'size': 10},
            yticklabels=cols,
            xticklabels=cols,
            cmap='coolwarm', vmin=-1, vmax = 1)

plt.show()

## Wydzielenie zbioru testowego




In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(columns='MEDV')
y = df['MEDV']

#Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

print('Ilość przypadków treningowych %d ' % len(X_train))
print('Ilość przypadków testowych    %d ' % len(X_test))

## Model dla pojedynczej zmiennej

In [ ]:
from sklearn.linear_model import LinearRegression

lr = linear_model.LinearRegression()
lr.fit(X_train[['RM']],  y_train)

print('Nachylenie: %.3f' % lr.coef_[0])
print('Punkt przecięcia: %.3f' % lr.intercept_)

NameError: ignored

In [ ]:
plt.scatter(X_train[['RM']], y_train, c='lightblue')

x_range = [X_train[['RM']].min().values, X_train[['RM']].max().values ]
plt.plot(x_range, lr.predict(x_range), color='red', linewidth=2) 

plt.xlabel('Uśredniona liczba pomieszczeń [RM]')
plt.ylabel('Cena w tysiącach dolarów [MEDV]')
plt.show()

In [ ]:
num_rooms = [[ 5 ]]
price = lr.predict(num_rooms)
print("Cena w tysiącach dolarów: %.3f" % price)

In [ ]:
# ocena modelu
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

y_train_pred = lr.predict(X_train[['RM']])
y_test_pred = lr.predict(X_test[['RM']])

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test  = mean_squared_error(y_test, y_test_pred)
print('MSE na próbkach uczących: %.3f, testowych: %.3f' % (mse_train, mse_test))

r2_train = r2_score(y_train, y_train_pred)
r2_test  = r2_score(y_test, y_test_pred)
print('Współczynnik R^2 dla danych uczących: %.3f, testowych: %.3f' % (r2_train, r2_test))

In [ ]:
wyniki = pd.DataFrame(columns =['MSE train', 'MSE test', 'R2 train', 'R2 test' ])
wyniki.loc['LR zmienna [RM]'] = [mse_train, mse_test, r2_train, r2_test ]
wyniki

## Model regresji liniowej dla wybranych zmiennych


In [ ]:
# wybierzmy 3 zmienne o najwiekszym wsp. korelacji
cols = ['LSTAT', 'PTRATIO', 'RM']

lr = linear_model.LinearRegression()
lr.fit(X_train[cols],  y_train)

y_train_pred = lr.predict(X_train[cols])
y_test_pred = lr.predict(X_test[cols])

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test  = mean_squared_error(y_test, y_test_pred)

r2_train = r2_score(y_train, y_train_pred)
r2_test  = r2_score(y_test, y_test_pred)

wyniki.loc['LR zmienne %s' % str(cols)] = [ mse_train, mse_test, r2_train, r2_test ]
wyniki

## Wykres residuów (wartości resztowe)

In [ ]:
plt.scatter(y_train_pred,  y_train_pred - y_train, c='blue', marker='o', label='Dane uczące')
plt.scatter(y_test_pred,  y_test_pred - y_test, c='lightgreen', marker='s', label='Dane testowe')
plt.xlabel('Przewidywane wartości')
plt.ylabel('Wartości resztowe')
plt.legend(loc='upper left')
plt.hlines(y=0, xmin=-10, xmax=50, lw=2, color='red')
plt.xlim([-10, 50])

plt.show()

## Model regresji dla wszystkich zmienneych

In [ ]:
lr = linear_model.LinearRegression()
lr.fit(X_train,  y_train)

y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test  = mean_squared_error(y_test, y_test_pred)

r2_train = r2_score(y_train, y_train_pred)
r2_test  = r2_score(y_test, y_test_pred)

wyniki.loc['LR wszystkie zmienne'] = [ mse_train, mse_test, r2_train, r2_test ]
wyniki

In [ ]:
# lasso dla wszystkich zmiennych 
from sklearn.linear_model import Lasso

lasso = Lasso()
lasso.fit(X_train, y_train)
y_train_pred = lasso.predict(X_train)
y_test_pred = lasso.predict(X_test)
print(lasso.coef_)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test  = mean_squared_error(y_test, y_test_pred)

r2_train = r2_score(y_train, y_train_pred)
r2_test  = r2_score(y_test, y_test_pred)

wyniki.loc['Lasso wszystkie zmienne'] = [ mse_train, mse_test, r2_train, r2_test ]
wyniki

In [ ]:
# silniejsza regularyzacja lasso
lasso = Lasso(alpha=0.1)
lasso.fit(X_train, y_train)
y_train_pred = lasso.predict(X_train)
y_test_pred = lasso.predict(X_test)
print(lasso.coef_)

wyniki.loc['Lasso alpha=0.1 wszystkie zmienne'] = [ mean_squared_error(y_train, y_train_pred), 
                                                   mean_squared_error(y_test, y_test_pred), 
                                                   r2_score(y_train, y_train_pred), 
                                                   r2_score(y_test, y_test_pred) ]
wyniki

## Modelowanie nieliniowych zależności

In [ ]:
# zmienna 'LSTAT'

sns.lmplot(data=df, x='LSTAT', y='MEDV')

In [ ]:
from sklearn.preprocessing import PolynomialFeatures 

# tworzy wielomianowe cechy
quadratic = PolynomialFeatures(degree=2)
X_quad_train = quadratic.fit_transform(X_train[['LSTAT']])

lr = LinearRegression()
lr.fit(X_quad_train, y_train)

y_train_pred = lr.predict(X_quad_train)
y_test_pred = lr.predict(quadratic.transform(X_test[['LSTAT']]))

wyniki.loc['LR kwadratowe (d=2) [LSTAT]'] = [ mean_squared_error(y_train, y_train_pred), 
                                              mean_squared_error(y_test, y_test_pred), 
                                              r2_score(y_train, y_train_pred), 
                                              r2_score(y_test, y_test_pred) ]
wyniki

In [ ]:
X_fit = np.arange(X_train[['LSTAT']].values.min(), X_train[['LSTAT']].values.max(), 1)[:, np.newaxis]
y_quad_fit = lr.predict(quadratic.transform(X_fit))

# tworzy wynikowy wykres
plt.scatter(X_train[['LSTAT']], y_train, label='Punkty uczące', color='orange')
plt.scatter(X_test[['LSTAT']], y_test, label='Punkty testowe', color='green')

plt.plot(X_fit, y_quad_fit, 
         label='Kwadratowe (d = 2)',
         color='red', 
         lw=2,
         linestyle='-')

plt.xlabel('Odsetek uboższej części społeczeństwa [LSTAT]')
plt.ylabel('Cena w tysiącach dolarów [MEDV]')
plt.legend(loc='upper right')

In [ ]:
# przekształcenie cechy (zmiennej) aby uzyskać liniowaą zalezność
X_log_train = np.log(X_train[['LSTAT']].values)

lr.fit(X_log_train, y_train)

lr = LinearRegression()
lr.fit(X_log_train, y_train)

y_train_pred = lr.predict(X_log_train)
y_test_pred = lr.predict(np.log(X_test[['LSTAT']]))

wyniki.loc['LR log([LSTAT])'] = [ mean_squared_error(y_train, y_train_pred), 
                                  mean_squared_error(y_test, y_test_pred), 
                                  r2_score(y_train, y_train_pred), 
                                  r2_score(y_test, y_test_pred) ]
wyniki

In [ ]:
X_fit = np.arange(X_log_train.min()-1, X_log_train.max()+1, 1)[:, np.newaxis]
y_fit = lr.predict(X_fit)

plt.scatter(X_log_train, y_train, label='Punkty uczące', color='orange')

plt.plot(X_fit, y_fit, 
         label='Liniowe (d = 1)', 
         color='blue', 
         lw=2)

plt.xlabel('log(odsetek uboższej części społeczeństwa [LSTAT])')
plt.ylabel('Cena w tysiącachdolarów [MEDV]')
plt.legend(loc='lower left')

## Zadanie 

Pod adresem https://www.fizyka.umk.pl/~grochu/wdm/files/diabetes.csv znajduje się plik zawierający dane `n=442` pacjentów chorych na cukrzycę (`diabetes`). Każdy przypadek opisany jest 10 zmiennymi numerycznymi:  wiek (`AGE`), płeć (`SEX`), wskaźnik masy ciała (`BMI`), średnie ciśnienie krwi (`BP`) i sześć pomiarów surowicy krwi (`S1 S2 S3 S4 S5 S6`). Ostatnia kolumna (zmienna `Y`) zawiera wartości określające stopnień zaawansowania choroby. Zadanie polega na zbudowaniu modeli regresji liniowej przewidujących wartość zmiennej `Y` zgodnie z poniższymi wytycznymi.

1. Wczytaj plik i sprawdź, czy w danych występują braki oraz sprawdź, czy wszystkie zmienne są wartościami numerycznymi. Jeżeli zajdzie potrzeba usuń obserwacje zawierające braki a zmienne przetransformuj do postaci numerycznej.

In [ ]:
import pandas as pd

df = pd.read_csv('https://www.fizyka.umk.pl/~grochu/wdm/files/diabetes.csv')

print(df)

df.info()

df.isna().any()

2. Podziel dane na dwie części: treningową zawierającą 75% przypadków i testową zawierającą pozostałe 25% przypadków.   
Modele regresji trenuj wyłacznie na części treningowej.  
Do podziału danych możesz wykorzystać funkcję [train_test_split()](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) z pakietu scikit-learn.

In [ ]:
from sklearn.model_selection import train_test_split

dfx = df.drop(columns = "Y")
dfy = df["Y"]

dfx_train, dfx_test, dfy_train, dfy_test = train_test_split(dfx, dfy, train_size = 0.75, test_size = 0.25, random_state=0)

print('Ilość przypadków treningowych %d ' % len(dfy_train))
print('Ilość przypadków testowych    %d ' % len(dfy_test))

3. Sporządź wykres parowy [pairplot()](https://seaborn.pydata.org/generated/seaborn.pairplot.html) zbioru treningowego i na jego podstawie wybierz jedną zmienną, która wydaje się posiadać liniową zależność względem zmiennej wyjściowej `Y`. Zbuduj model regresji liniowej dla wybranej zmiennej i wyznacz błąd MSE predykcji `Y` na zbiorze treningowym oraz na zbiorze testowym.

In [ ]:
import seaborn as sns
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

#sns.pairplot(df)

lr = LinearRegression()
lr.fit(dfx_train[['BMI']], dfy_train)

print('Nachylenie: %.3f' % lr.coef_[0])
print('Punkt przecięcia: %.3f' % lr.intercept_)

plt.scatter(dfx_train[['BMI']], dfy_train, c='lightblue')

x_range = [dfx_train[['BMI']].min().values, dfx_train[['BMI']].max().values ]
plt.plot(x_range, lr.predict(x_range), color='red', linewidth=2) 

plt.xlabel('BMI')
plt.ylabel('Y')
plt.show()

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

dfy_train_pred = lr.predict(dfx_train[['BMI']])
dfy_test_pred = lr.predict(dfx_test[['BMI']])

mse_train = mean_squared_error(dfy_train, dfy_train_pred)
mse_test  = mean_squared_error(dfy_test, dfy_test_pred)
print('MSE na próbkach uczących: %.3f, testowych: %.3f' % (mse_train, mse_test))

r2_train = r2_score(dfy_train, dfy_train_pred)
r2_test  = r2_score(dfy_test, dfy_test_pred)
print('Współczynnik R^2 dla danych uczących: %.3f, testowych: %.3f' % (r2_train, r2_test))

res = pd.DataFrame(columns = ['MSE train', 'MSE test', 'R2 train', 'R2 test'])
res.loc['LR zmienna [BMI]'] = [mse_train, mse_test, r2_train, r2_test]
res



4. Zbuduj model regresji liniowej wielowymiarowej uwzględniając wszystkie zmienne do opisu zmiennej wyjściowej `Y`. Zastosuj w tym celu jedną, wybraną metodę z regularyacją (np. grzbietową lub Lasso). Przeprowadź obliczenia dla przynajmniej 2 róznych wartości współczynnika określającego siłę regularyzacji $\alpha$ i porównaj wyniki. 

In [ ]:
res = pd.DataFrame(columns = ['MSE train', 'MSE test', 'R2 train', 'R2 test'])

lasso = Lasso(alpha=0.1)
lasso.fit(dfx_train, dfy_train)
dfy_train_pred = lasso.predict(dfx_train)
dfy_test_pred = lasso.predict(dfx_test)
print(lasso.coef_)

res.loc['Lasso alpha=0.1 wszystkie zmienne'] = [ mean_squared_error(dfy_train, dfy_train_pred), mean_squared_error(dfy_test, dfy_test_pred), r2_score(dfy_train, dfy_train_pred), r2_score(dfy_test, dfy_test_pred) ]
res

lasso = Lasso(alpha=0.3)
res.loc['Lasso alpha=0.3 wszystkie zmienne'] = [ mean_squared_error(dfy_train, dfy_train_pred), mean_squared_error(dfy_test, dfy_test_pred), r2_score(dfy_train, dfy_train_pred), r2_score(dfy_test, dfy_test_pred) ]
res

lasso = Lasso(alpha=0.5)
res.loc['Lasso alpha=0.5 wszystkie zmienne'] = [ mean_squared_error(dfy_train, dfy_train_pred), mean_squared_error(dfy_test, dfy_test_pred), r2_score(dfy_train, dfy_train_pred), r2_score(dfy_test, dfy_test_pred) ]
res

5. Spośród stworzonych modeli regresji wybierz najlepszy (ten o najmniejszym MSE na zbiorze testowym) i wypisz (lub wyświetl) jego współczynniki regresji.

In [ ]:
lasso = Lasso(alpha=0.1)
res = pd.DataFrame(columns = ['R2 train', 'R2 test'])
res.loc['Lasso alpha=0.1 wszystkie zmienne'] = [r2_score(dfy_train, dfy_train_pred), r2_score(dfy_test, dfy_test_pred) ]
res

,R2 train,R2 test
Lasso alpha=0.1 wszystkie zmienne,0.555354,0.357998
